In [1]:
with open('data/input.txt', 'r', encoding='utf-8') as f:
    shakespeare = f.read()

In [2]:
print(f'length of the input text for pretraining: {len(shakespeare)}.')
print(shakespeare[:1000])

length of the input text for pretraining: 1115394.
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak thi

In [9]:
# List all unique characters that occurs in the input text
chars = sorted(list(set(shakespeare)))
vocab_size = len(chars)
print('ALL chars:' + ''.join(chars))
print()
print(f'TOTAL CHARS NUM: {vocab_size}')

ALL chars:
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz

TOTAL CHARS NUM: 65


In [11]:
# mapping from characters to integers for encoding
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s : [stoi[c] for c in s] # Take a string and output a list of integers
decode = lambda i : ''.join([itos[c] for c in i]) # Take a list of integers and output a list of string

test_string = "hii, there"
print(encode(test_string))
print(decode(encode(test_string)))

[46, 47, 47, 6, 1, 58, 46, 43, 56, 43]
hii, there


In [13]:
# encode the entire text dataset and store it into a torch.Tensor
import torch

data = torch.tensor(encode(shakespeare), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([1115394]) <built-in method type of Tensor object at 0x0000022893F60040>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61, 

In [14]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [15]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'input: {context}, target: {target}')

input: tensor([18]), target: 47
input: tensor([18, 47]), target: 56
input: tensor([18, 47, 56]), target: 57
input: tensor([18, 47, 56, 57]), target: 58
input: tensor([18, 47, 56, 57, 58]), target: 1
input: tensor([18, 47, 56, 57, 58,  1]), target: 15
input: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
input: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58


In [19]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will be process in parallel?
block_size = 8 # What is the maximum context length for predictions?

In [26]:
ix = torch.randint(len(data) - block_size, (batch_size,))
ix

tensor([ 619016, 1042503,  342314, 1011765])

In [27]:
torch.stack([data[i:i+block_size] for i in ix])

tensor([[ 5, 43, 56,  1, 57, 46, 39, 50],
        [61, 39, 57,  1, 40, 59, 56, 52],
        [59, 58,  1, 44, 39, 47, 52, 58],
        [57, 58, 47, 52, 45,  8,  0,  0]])

In [28]:
torch.stack([data[i+1:i+block_size+1] for i in ix])

tensor([[43, 56,  1, 57, 46, 39, 50, 50],
        [39, 57,  1, 40, 59, 56, 52, 58],
        [58,  1, 44, 39, 47, 52, 58, 50],
        [58, 47, 52, 45,  8,  0,  0, 28]])

In [30]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[52, 57,  8,  0, 21,  5, 50, 50],
        [47, 58, 46,  1, 63, 53, 59,  6],
        [ 1, 53, 40, 43, 63, 12,  0, 26],
        [50, 42, 57, 58,  6,  0, 32, 46]])
targets:
torch.Size([4, 8])
tensor([[57,  8,  0, 21,  5, 50, 50,  1],
        [58, 46,  1, 63, 53, 59,  6,  1],
        [53, 40, 43, 63, 12,  0, 26, 39],
        [42, 57, 58,  6,  0, 32, 46, 43]])


In [32]:
for b in range(batch_size): # batch dimension
    print(f'batch {b+1}/{batch_size}')
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")
    print()

batch 1/4
when input is [52] the target: 57
when input is [52, 57] the target: 8
when input is [52, 57, 8] the target: 0
when input is [52, 57, 8, 0] the target: 21
when input is [52, 57, 8, 0, 21] the target: 5
when input is [52, 57, 8, 0, 21, 5] the target: 50
when input is [52, 57, 8, 0, 21, 5, 50] the target: 50
when input is [52, 57, 8, 0, 21, 5, 50, 50] the target: 1

batch 2/4
when input is [47] the target: 58
when input is [47, 58] the target: 46
when input is [47, 58, 46] the target: 1
when input is [47, 58, 46, 1] the target: 63
when input is [47, 58, 46, 1, 63] the target: 53
when input is [47, 58, 46, 1, 63, 53] the target: 59
when input is [47, 58, 46, 1, 63, 53, 59] the target: 6
when input is [47, 58, 46, 1, 63, 53, 59, 6] the target: 1

batch 3/4
when input is [1] the target: 53
when input is [1, 53] the target: 40
when input is [1, 53, 40] the target: 43
when input is [1, 53, 40, 43] the target: 63
when input is [1, 53, 40, 43, 63] the target: 12
when input is [1, 53, 

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

In [59]:
# A basic neural network for bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        # the embedding table is in the shape of (vocab_size, vocab_size)
        #   a b c ... z
        # a 
        # b
        # .
        # z
        # each token directly reads off the logits for the next token from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, target=None):
        # idx and target are in the size of (B, T)
        logits = self.token_embedding_table(idx) # (B, T, vocab_size)
        
        if target is None: # for generation without providing target
            loss = None
        else:
            # Reshape the logits tensor to meet definition of the cross_entropy function in Pytorch
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # Concatenate B, T
            target = target.view(B*T)    # Same reshaping to the target tensor
            
            # calculate the loss
            loss = F.cross_entropy(logits, target)
        
        return logits, loss
    
    def generation(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self(idx)
            
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            
        return idx
    
n = BigramLanguageModel(vocab_size)
out, loss = n(xb, yb)
print(out.shape)
print(loss)
print()

logits = n(torch.zeros((1, 1), dtype=torch.long))[0] # (B, T) = (1, 1)
print(logits)  # logits size (1, 65, 1)
print(logits[:, -1, :]) # focus only on the final step (1, 65)
probs = F.softmax(logits, dim=-1)
print(probs)
next_idx = torch.multinomial(probs[0], num_samples=1)[0].tolist()
print(next_idx) # (1, 1)
print(decode(next_idx))

torch.Size([32, 65])
tensor(4.7908, grad_fn=<NllLossBackward0>)

tensor([[[-1.2518, -0.4433, -0.9142,  0.2837,  0.1342, -0.5358, -1.0149,
           1.2734, -0.4987,  1.4899, -0.5600, -2.0031,  1.0201,  0.5014,
           1.3083,  0.9775, -0.8054,  0.0706, -1.5133,  0.4046, -0.0877,
           0.6163,  0.0603, -1.1521,  0.3436,  1.2332,  1.2264,  0.1541,
           0.1684,  0.4070, -0.3775,  0.2763,  0.5255,  1.8775, -0.1354,
          -0.8933,  1.0721, -0.6513,  0.3540,  0.8912, -0.0789,  1.0509,
          -1.1171,  0.4218,  0.3554,  0.1551, -0.3013,  0.9438,  0.2701,
          -1.4606,  1.4467, -0.8813, -0.6861,  1.6792,  0.1064,  0.1194,
           1.0806, -0.3393, -1.1097, -0.5630, -0.5275, -0.7349,  0.4303,
          -0.1852, -0.4885]]], grad_fn=<EmbeddingBackward0>)
tensor([[-1.2518, -0.4433, -0.9142,  0.2837,  0.1342, -0.5358, -1.0149,  1.2734,
         -0.4987,  1.4899, -0.5600, -2.0031,  1.0201,  0.5014,  1.3083,  0.9775,
         -0.8054,  0.0706, -1.5133,  0.4046, -0.0877,  

In [61]:
print(decode(n.generation(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


3etsu
&:S;Q?'XHXPa3buWxT-P.PA;CPGj&F-cMs-Y$z:THykqDODooJ?kXxowrqMiIOpVjyIptpbvOUhB.&-kACaHwawJxTcQaD
